1 — Kill the last inner loop: simulate_financing

In [ ]:
def simulate_financing_vec(
    T, n_scenarios,  # n_scenarios = how many “what-if” grids you’ll run
    mean, sigma, spike_prob, spike_factor, rng=None
):
    """
    Vectorised jump-diffusion financing spreads.
    Returns shape (n_scenarios, T)
    """
    rng = np.random.default_rng() if rng is None else rng

    base = rng.normal(mean, sigma, size=(n_scenarios, T))
    jumps = rng.random((n_scenarios, T)) < spike_prob          # 0/1 mask
    jump_size = jumps * (spike_factor * sigma)
    out = np.clip(base + jump_size, 0.0, None)                 # no negatives
    return out


The old version looped 12 times per call – trivial for one path, but you call it three times for every grid node. This single call replaces them all. 

2 — Pre-compute the Monte-Carlo cube once

In [ ]:
rng  = np.random.default_rng(seed)
sim_draws = rng.standard_normal((N_SIMULATIONS, N_MONTHS, 4))  # idx, H, E, M
L   = np.linalg.cholesky(cov_mat       / 12)  # annual → monthly
mu  = np.array([mu_idx, mu_H, mu_E, mu_M]) / 12

# Shape: (n_sim, n_months, 4)
alpha_paths = sim_draws @ L.T + mu

That already existed in your snippet, but you rebuilt it inside the grid loops. Lift it out—every (ext_pct, act_pct) pair re-uses the same stochastic universe.

3 — Vectorise the (ext %, act %) grid itself

In [ ]:
ext_step = ext_step_pct
ext_grid = np.arange(0.0, max_external_combined + 1e-9, ext_step)

# Build all legal pairs where 0 ≤ act_pct ≤ ext_pct
ext_mesh, act_mesh = np.meshgrid(ext_grid, ext_grid)
mask = act_mesh <= ext_mesh
ext_vec = ext_mesh[mask]                     # 1-D of length G
act_vec = act_mesh[mask]

# Derived shares
E_pct = ext_vec - act_vec                    # Ext-PA share
A_pct = act_vec                              # Active-Ext share
Z_pct = 1.0 - ext_vec                        # Internal share


Now G = len(ext_vec) is your “scenario” dimension. We’ll broadcast everything over it.

4 — Vectorised financing draws for all grid nodes
python
Copy


In [ ]:
f_int   = simulate_financing_vec(N_MONTHS, G,
                                 internal_financing_mean,
                                 internal_financing_sigma,
                                 internal_spike_prob,
                                 internal_spike_factor,
                                 rng)

f_ext   = simulate_financing_vec(N_MONTHS, G,
                                 ext_pa_financing_mean,
                                 ext_pa_financing_sigma,
                                 ext_pa_spike_prob,
                                 ext_pa_spike_factor,
                                 rng)

f_act   = simulate_financing_vec(N_MONTHS, G,
                                 act_ext_financing_mean,
                                 act_ext_financing_sigma,
                                 act_ext_spike_prob,
                                 act_ext_spike_factor,
                                 rng)


Result: three arrays, each shape (G, T) instead of thousands of tiny 12-element vectors produced in nested loops.

5 — Broadcast the capital weights into the Monte-Carlo cube

In [ ]:
#  Capital in *millions*, shape (G, 3)
caps_mm = np.column_stack([E_pct, A_pct, Z_pct]) * total_fund_capital

# Expand to (G, n_sim, n_months, 3) for broadcasting
caps4 = caps_mm[:, None, None, :]            # adds two singleton axes

# Slice the alpha paths into the three sleeves.
#   0 = index hedge, 1 = In-House H, 2 = Alpha-Ext E, 3 = External M
#   (H and Z share the In-House path here, but you may treat Z as “cash” = 0.)
H_path = alpha_paths[..., 1]                 # shape (n_sim, n_months)
E_path = alpha_paths[..., 2]
M_path = alpha_paths[..., 3]

# Stack to align: shape (1, n_sim, n_months, 3)
ret_stack = np.stack([E_path, M_path, H_path], axis=-1)[None, ...]

# Gross $-returns before financing: (G, n_sim, n_months)
gross_ret = (caps4 * ret_stack).sum(axis=-1) / total_fund_capital


6 — Net out financing in one line

No loops, just pure broadcasting.

7 — Aggregate statistics

In [ ]:
# mean & vol by scenario and simulation
ann_mean = net_ret.mean(axis=-1) * 12
ann_vol  = net_ret.std(axis=-1, ddof=1) * np.sqrt(12)

# Collapse MC dimension → expect, VaR, whatever you need
mean_of_mean = ann_mean.mean(axis=1)         # shape (G,)
p05         = np.percentile(ann_mean, 5, axis=1)


The resulting vectors (G long) are what you used to build results = {"Base": …} earlier, but now they arrive with no explicit for loops anywhere.